In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.0001
Capacitance = 1.

time_limit = 100
ps = 10
psm = 1

solver = "gp_canonical_X"
# solver = "gp"

# Ciropt problem

In [3]:
problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)

res, model, sp_exp = problem.solve(solver=solver, verbose=True, debug=True, time_limit=time_limit, psm=psm, ps=ps)[:3]
print(res)

dim_G=5, dim_F=4
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>


/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:495: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  model.addConstr(  (M1 @ var_x).item()  - gp_trace(M2 @ var_X) == 0)


<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>


/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:509: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  + gp_trace(M2 @ var_X) \
/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:510: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  - gp_trace(M3 @ var_X) == 0)
/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:525: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  model.addConstr( gp_trace(M1 @ var_X) + (M2 @ var_x).item() == 0)


Set parameter NonConvex to value 2
Set parameter TimeLimit to value 100
Set parameter FeasibilityTol to value 1e-09
Set parameter Method to value 0
Set parameter PoolSearchMode to value 1
Set parameter Heuristics to value 0.001
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25 rows, 47 columns and 25 nonzeros
Model fingerprint: 0xc31bf274
Model has 43 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e-05, 1e+00]
  QLMatrix range   [5e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+03, 1e+03]
  RHS range        [1e+00, 1e+00]
Presolve removed 25 rows and 2 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 21 rows and 2 columns
Presolve time: 0.00s
Presolved: 378 rows, 123 columns, 1079 nonzeros
Presolved model has 95 bilinear constraint(s)
Variable t

In [4]:
# model.tune()

# for i in range(model.tuneResultCount):
#     model.getTuneResult(i)
#     model.write('tune'+str(i)+'.prm')

# gp_vars = {name : var_x[idx].item() for name, idx in problem.name2idx.items()}
# co.gp_print_solutions(model, gp_vars, all=False)

In [6]:
print(model.printQuality())


Solution quality statistics for model Unnamed :
  Maximum violation:
    Bound       : 0.00000000e+00
    Constraint  : 1.89779305e-08 (QC22)
    Integrality : 0.00000000e+00
None


In [8]:
res

{'b': 0.9999999970168865,
 'h': 2.362919398175109,
 'd': 1000.0,
 'alpha': 0.4231896162011234,
 'beta': 0.4231517686761676}

# PEP verification

In [9]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step

b = res["b"] 
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [10]:
problem = PEPit.PEP()
func = problem.declare_function(
                    SmoothStronglyConvexFunction,
                    mu=mu, 
                    L=L_smooth) 


x_P_star = func.stationary_point()
g_star, f_star = func.oracle(x_P_star)
x_P_1 = problem.set_initial_point()

g_1, f_1 = func.oracle(x_P_1)
x_P_1p5 = x_P_1 - (alpha * h / Capacitance) * g_1 
g_1p5, f_1p5 = func.oracle(x_P_1p5)
x_P_2 = x_P_1  - (beta * h / Capacitance) * g_1 - ((1 - beta) * h / Capacitance) * g_1p5

g_2, f_2 = func.oracle(x_P_2)

# Set the performance metric to the function values accuracy
E_1 = (Capacitance/2) * (x_P_1 - x_P_star)**2
E_2 = (Capacitance/2) * (x_P_2 - x_P_star)**2
Delta_1 = b * (f_1 - f_star)
problem.set_performance_metric(E_2 - (E_1 - Delta_1))

# Solve the PEP with verbose = 1
verbose = 0
diff = problem.solve(verbose=verbose, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.3067190662852113e-09
